In [1]:
import json
import time

import sys
sys.path.append("../..")
from utils.preprocessing import readAndFilterEvent, readAndFilterFile


In [2]:
event_normalized = """{"process.parent.process.pid": "1","auditd.data.arch": "x86_64","Headers.VmId": "e708da01-58f5-4f8c-8923-cc64a5d52c83","Headers.SubscriptionId": "1c8adc77-05c9-45fb-976c-e90c4bea200f","Role": "pstn","process.ppid": "1","hostname": "neu-qa-pstn-pm1","rule.sidid": "execution","Headers.Workload": "LinuxSkypePSTN","process.parent.process.title": "/sbin/init","TimeStamp": "2022-10-31T12:02:02.6110000Z","auditd.data.a1": "55bff0690480","process.parent.process.name": "systemd","Headers.CustomerName": "PSTNInfrastructure","auditd.data.a2": "55bff063d900","full_log": "/bin/bash","Headers.ResourceGroupName": "neu-pstn-skypeout-ppe","process.args": "/bin/bash /etc/init.d/mdsd stop","Rule": "auditbeat","auditd.data.tty": "(none)","Latency": "1033028","auditd.message_type": "syscall","auditd.data.a3": "55bff06fdd00","Headers.Role": "pstn","process.parent.process.executable": "/lib/systemd/systemd","process.parent.process.ppid": "0","auditd.data.exit": "0","id": "509947759","process.title": "/bin/bash /etc/init.d/mdsd stop","auditd.data.syscall": "execve","auditd.summary.object.primary": "/etc/init.d/mdsd","process.name": "mdsd","process.executable": "/bin/bash","auditd.result": "success","auditd.summary.actor.secondary": "root","process.working_directory": "/","auditd.sequence": "509947759","auditd.summary.object.type": "file","process.parent.process.start_time": "2022-07-19T10:57:54.7300000Z","program_name": "auditbeat","auditd.summary.actor.primary": "unset","auditd.summary.how": "/bin/bash","auditd.data.argc": "3","auditd.data.a0": "55bff06fdca0","sidid": "execution","process.pid": "11233"}"""
event_hierarchical = """{"process":{"parent":{"process":{"pid":"23463","title":"/bin/bash/<appcontainer>","name":"bash","executable":"/bin/bash","ppid":"23459","start_time":"2022-06-13T07:37:19.1800000Z"}},"ppid":"23463","args":"ln-sf/mnt/resource/hadoop/yarn/local/usercache/livy/filecache/50757218/scala-logging_2.11-3.9.0.jarscala-logging_2.11-3.9.0.jar","title":"ln-sf/mnt/resource/hadoop/yarn/local/usercache/livy/filecache/50757218/scala-logging_2.11-3.9.0.jarscala-logging_2.11-3.9.0.j","name":"ln","executable":"/bin/ln","working_directory":"/<appcontainer>","pid":"23650"},"auditd":{"data":{"arch":"x86_64","a1":"55daa78c1040","a2":"55daa78b79f0","tty":"(none)","a3":"8","exit":"0","syscall":"execve","argc":"4","a0":"55daa78b2d90"},"message_type":"syscall","summary":{"object":{"primary":"/bin/ln","type":"file"},"actor":{"secondary":"yarn","primary":"unset"},"how":"/bin/ln"},"result":"success","sequence":"2222174356"},"Headers":{"VmId":"efcb7627-4981-40f3-809a-fdd82771f7ed","SubscriptionId":"32abfb07-f763-4cb7-bad4-a13fdfd7e9db","Workload":"DataInsights","CustomerName":"DataInsights","ResourceGroupName":"rg0-7ff91af554614e2ab9465f7f5bc3686aresourcegroup","ClusterName":"o365ipdinor01-sp-ne01","Role":"WorkerNode"},"Role":"WorkerNode","hostname":"wn23-o365ip","rule":{"sidid":"execution"},"TimeStamp":"2022-06-13T07:37:19.5880000Z","full_log":"/bin/ln","Rule":"auditbeat","Latency":"125739","id":"2222174356","program_name":"auditbeat","sidid":"execution"}"""

try:
    json.loads(event_normalized)
    json.loads(event_hierarchical)
except json.decoder.JSONDecodeError as e:
    print("Invalid JSON: {}".format(e))
else:
    print("Valid JSONs")

Valid JSONs


In [3]:
from utils.constants import AUDITD_FIELDS, AUDITD_TYPES
a = readAndFilterEvent(event_normalized, normalizedFields=AUDITD_FIELDS, filterDict={"rule.sidid": AUDITD_TYPES})
b = readAndFilterEvent(event_hierarchical, normalizedFields=AUDITD_FIELDS, jsonType = "nested", filterDict={"rule.sidid": AUDITD_TYPES})
print(type(a), a.shape, type(b), b.shape)

<class 'pandas.core.frame.DataFrame'> (1, 14) <class 'pandas.core.frame.DataFrame'> (1, 14)


In [4]:
b

,TimeStamp,rule.sidid,hostname,auditd.data.syscall,auditd.summary.actor.primary,auditd.summary.actor.secondary,auditd.summary.object.primary,process.title,process.args,process.working_directory,process.pid,process.ppid,process.parent.process.executable,process.parent.process.title
0,2022-06-13T07:37:19.5880000Z,execution,wn23-o365ip,execve,unset,yarn,/bin/ln,ln-sf/mnt/resource/hadoop/yarn/local/usercache...,ln-sf/mnt/resource/hadoop/yarn/local/usercache...,/<appcontainer>,23650,23463,/bin/bash,/bin/bash/<appcontainer>


In [5]:
a

,TimeStamp,rule.sidid,hostname,auditd.data.syscall,auditd.summary.actor.primary,auditd.summary.actor.secondary,auditd.summary.object.primary,process.title,process.args,process.working_directory,process.pid,process.ppid,process.parent.process.executable,process.parent.process.title
0,2022-10-31T12:02:02.6110000Z,execution,neu-qa-pstn-pm1,execve,unset,root,/etc/init.d/mdsd,/bin/bash /etc/init.d/mdsd stop,/bin/bash /etc/init.d/mdsd stop,/,11233,1,/lib/systemd/systemd,/sbin/init


#### Benchmark

In [6]:
tick1 = time.time()
filename = r"C:\Users\dtrizna\Code\nebula\data\auditd_msft_raw\1662470678.json"
df = readAndFilterFile(filename, normalizedFields=AUDITD_FIELDS, filterDict={"rule.sidid": AUDITD_TYPES})
tick2 = time.time()
mean100tick1 = (tick2 - tick1)*100/df.shape[0]
print("Time taken: ~{:.8f} seconds per 100 events".format(mean100tick1))

Time taken: ~0.00213889 seconds per 100 events


In [7]:
import orjson
from pandas import concat

tick1 = time.time()
filename = r"C:\Users\dtrizna\Code\nebula\data\auditd_msft_raw\1662470678.json"
with open(filename, "rb") as f:
    data = orjson.loads(f.read())
dfs = []
for i, event in enumerate(data[0:10000]):
    dfs.append(readAndFilterEvent(event, normalizedFields=AUDITD_FIELDS, filterDict={"rule.sidid": AUDITD_TYPES}))
df = concat(dfs)
tick2 = time.time()
mean100tick2 = (tick2 - tick1)*100/df.shape[0]
print("Time taken: ~{:.8f} seconds per 100 events".format(mean100tick2))

print(f"Per event parsing is {mean100tick2/mean100tick1:.2f} times slower")

Time taken: ~0.11892068 seconds per 100 events
Per event parsing is 55.60 times slower


Uff, rougly 50 times slower if reading event by event...

In [8]:
with open("test.json", "w") as f:
    f.write(df.to_json(orient='records', indent=4))

### Normalization

IP

In [9]:
col = 'auditd.summary.object.primary'

In [13]:
from numpy import where

def labelIPs(ldf, 
              col = 'auditd.summary.object.primary'):
    
    ldf[col] = where(ldf[col].str.startswith('127.'), "(lopIP)", ldf[col])
    ldf[col] = where(ldf[col].str.match('169.254.169.254'), "(imds)", ldf[col])
    ldf[col] = where(ldf[col].str.match(r'(^10\.)|(^172\.1[6-9]\.)|(^172\.2[0-9]\.)|(^172\.3[0-1]\.)|(^192\.168\.)'), "(prvIP)", ldf[col])
    ldf[col] = where(ldf[col].str.match(r'^[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}$'), "(pubIP)", ldf[col])
    ldf[col] = where(ldf[col].str.match(r'^([0-9a-fA-F]{1,4}:){1,7}:'), "(IPv6)", ldf[col])
    return ldf

print(labelIPs(df)[col].isin(["(lopIP)", "(imds)", "(prvIP)", "(pubIP)", "(IPv6)"]).sum())

import time
tick1 = time.time()
labelIPs(df)[col].value_counts()
tick2 = time.time()
delta1 = tick2 - tick1
print("Time taken: {:.8f} seconds".format(tick2 - tick1))

928
Time taken: 0.02862644 seconds


In [14]:
def labelIPs(ldf, 
              col = 'auditd.summary.object.primary'):
    
    ldf[col] = ldf[col].str.replace(r'127\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}', "(lopIP)", regex=True)
    ldf[col] = ldf[col].str.replace(r'169\.254\.169\.254', "(imds)", regex=True)
    ldf[col] = ldf[col].str.replace(r'(^10\.)|(^172\.1[6-9]\.)|(^172\.2[0-9]\.)|(^172\.3[0-1]\.)|(^192\.168\.)', "(prvIP)", regex=True)
    ldf[col] = ldf[col].str.replace(r'[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}', "(pubIP)", regex=True)
    ldf[col] = ldf[col].str.replace(r'([0-9a-fA-F]{1,4}:){1,7}:', "(IPv6)", regex=True)
    return ldf

print(labelIPs(df)[col].isin(["(lopIP)", "(imds)", "(prvIP)", "(pubIP)", "(IPv6)"]).sum())

import time
tick1 = time.time()
labelIPs(df)[col].value_counts()
tick2 = time.time()
delta1 = tick2 - tick1
print("Time taken: {:.8f} seconds".format(tick2 - tick1))

928
Time taken: 0.05752587 seconds


In [15]:
import re
def labelIPString(string):
    string = re.sub(r'127\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}', "(lopIP)", string)
    string = re.sub(r'169\.254\.169\.254', "(imds)", string)
    string = re.sub(r'(^10\.)|(^172\.1[6-9]\.)|(^172\.2[0-9]\.)|(^172\.3[0-1]\.)|(^192\.168\.)', "(prvIP)", string)
    string = re.sub(r'^[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}$', "(pubIP)", string)
    string = re.sub(r'^([0-9a-fA-F]{1,4}:){1,7}:', "(IPv6)", string)
    return string

df[col].apply(labelIPString).isin(["(lopIP)", "(imds)", "(prvIP)", "(pubIP)", "(IPv6)"]).sum()

928

In [16]:
import time
tick1 = time.time()
df[col].apply(labelIPString).value_counts()
tick2 = time.time()
delta2 = tick2 - tick1
print("Time taken: {:.8f} seconds".format(tick2 - tick1))

print(f"Per event parsing is {delta2/delta1:.2f} times slower")

Time taken: 0.05805945 seconds
Per event parsing is 1.01 times slower


Domains

In [27]:

col='process.args'
df = readAndFilterFile(filename, normalizedFields=AUDITD_FIELDS, filterDict={"rule.sidid": AUDITD_TYPES})
df.fillna("(none)", inplace=True)

def labelStringDomain(string,
                    domain_regex = r"\b([a-zA-Z0-9][a-zA-Z0-9-]{0,61}[a-zA-Z0-9]\.)+(com|net|cz|io)\b",
                    placeholder = "(domain)"):
    return re.sub(domain_regex, placeholder, string)

tick1 = time.time()
newdf1 = df[col].apply(labelStringDomain)
tick2 = time.time()
delta1 = tick2 - tick1
print("String wise operation takes: {:.8f} seconds".format(delta1))

# =======================

df = readAndFilterFile(filename, normalizedFields=AUDITD_FIELDS, filterDict={"rule.sidid": AUDITD_TYPES})
df.fillna("(none)", inplace=True)


def labelStringTable(ldf, col,
                    domain_regex = r"([a-zA-Z0-9][a-zA-Z0-9-]{0,61}[a-zA-Z0-9]\.)+(com|net|cz|io)\b",
                    placeholder = "(domain)"):
        ldf[col] = where(ldf[col].str.match(domain_regex), placeholder, ldf[col])
        return ldf

tick1 = time.time()
newdf2 = labelStringTable(df, col)[col]
delta2 = tick2 - tick1
print("Table wise operation takes: {:.8f} seconds".format(delta2))

print(f"String wise operation is {delta1/delta2:.2f} times slower")

String wise operation takes: 0.08603954 seconds
Table wise operation takes: -0.75531602 seconds
String wise operation is -0.11 times slower


In [28]:
newdf1

0                              grep -qi ImageStatus failed
1        timeout 60 ctr --namespace (domain) container ...
2                  ctr --namespace (domain) container list
3                                                 sleep 10
4        /bin/bash /etc/node-problem-detector.d/plugin/...
                               ...                        
29540                           basename /usr/sbin/service
29541                           basename /usr/sbin/service
29542        systemctl --quiet is-active multi-user.target
29543            systemctl status slave-ha-service.service
29544                                               (none)
Name: process.args, Length: 29507, dtype: object